<a href="https://colab.research.google.com/github/EuniceOnduso/Intent-Classifier-Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set up the environment
!pip install -q scikit-learn pandas numpy

In [2]:
# install Gradio
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.7 MB/s eta 0:00:00


In [3]:
# prepare dataset
import pandas as pd
data = {
    "text": [
    "hi", "hello", "hey",
    "i want to order a pizza", "can i get a pizza",
    "what's the weather", "is it raining today",
    "goodbye", "see you later"
    ],

  "intent": [
    "greeting", "greeting", "greeting",
    "order_pizza", "order_pizza",
    "ask_weather", "ask_weather",
    "goodbye", "goodbye"
  ]
}
df = pd.DataFrame(data)
df

,text,intent
0,hi,greeting
1,hello,greeting
2,hey,greeting
3,i want to order a pizza,order_pizza
4,can i get a pizza,order_pizza
5,what's the weather,ask_weather
6,is it raining today,ask_weather
7,goodbye,goodbye
8,see you later,goodbye


In [4]:
# Preprocess data
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

x = df["text"]
y = df["intent"]

vectorizer = TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x_vect, y_encoded, test_size=0.2, random_state=42)

In [5]:
# train a classifer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5


In [6]:
# build the chatbot function
def predict_intent(text):
  text_vect = vectorizer.transform([text])
  pred = clf.predict(text_vect)[0]
  intent = encoder.inverse_transform([pred])[0]
  return intent

def chatbot_response(user_input):
  intent = predict_intent(user_input)

  responses = {
     "greeting": "Hey there! How can I help you?",
     "order_pizza": "Sure! What kind of pizza would you like?",
     "ask_weather": "It's sunny and warm today!",
     "goodbye": "Goodbye! Have a great day!"
  }

  return responses.get(intent, "i'm not sure how to help with that.")
  # try it out
chatbot_response("can i order a pizza")

'Sure! What kind of pizza would you like?'

In [9]:
# add the gradio interface
import gradio as gr

iface = gr.Interface (
     fn=chatbot_response,
     inputs=gr.Textbox(lines=2, placeholder='Ask me something...'),
     outputs="text",
     title = "Intent Classifier chatbot",
     description = "This is a simple chatbot that classifies your intent and gives a canned response."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e8c4f88ee9da72162.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
